In [1]:
import numpy as np
from scipy.stats import multivariate_normal

import emcee
import tqdm

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, 0.0, -inf, 0.0, 0.0, -inf, 0.0, 0.0, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, -inf, 0.0, 0.0, -inf, 0.0, -inf, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, -inf, -inf, 0.0, 0.0, 0.0, -inf, 0.0, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, -inf, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, 0.0, 0.0, 0.0, -inf, 0.0, 0.0, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, -inf, -inf, 0.0, 0.0, 0.0, -inf, 0.0, -inf, -inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

AttributeError: 'module' object has no attribute 'PTSampler'

In [ ]:
def triangle_llh(theta):
    fr = theta[-3:]
    fr_bf = (0.5, 0.5, 0.0)
    sigma = 0.2
    cov_fr = np.identity(3) * sigma
    return -np.log10(multivariate_normal.pdf(fr, mean=fr_bf, cov=cov_fr))

In [ ]:
print triangle_llh((0.2, 0.3, 0.5))
print triangle_llh((0.5, 0.5, 0.0))

In [ ]:
def lnprior(theta):
    ue1, ue2, ue3, um1, um2, um3, ut1, ut2, ut3, fr1, fr2, fr3 = theta
    
    allow = True
    # Flavour ratio bounds
    if 0. <= fr1 <= 1.0 and 0. <= fr2 <= 1.0 and 0. <= fr3 <= 1.0:
        pass
    else: allow = False

    # mixing elements 3sigma bound using nufit
    if 0.800 <= ue1 <= 0.844 and 0.515 <= ue2 <= 0.581 and 0.139 <= ue3 <= 0.155 \
    and 0.229 <= um1 <= 0.516 and 0.438 <= um2 <= 0.699 and 0.614 <= um3 <= 0.790 \
    and 0.249 <= ut1 <= 0.528 and 0.462 <= ut2 <= 0.715 and 0.595 <= ut3 <= 0.776:
        pass
    else: allow = False
        
    # TODO(shivesh): enforce sum of flavour ratios? enforce unitarity?
    if allow: return 0.
    else: return -np.inf

In [ ]:
def lnprob(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + triangle_llh(theta)

In [ ]:
ndim = 12
nwalkers = 200
ntemps = 1
burnin = 100
betas = np.array([1e0, 1e-1, 1e-2, 1e-3, 1e-4])
p0_base = [0.82, 0.55, 0.14, 0.40, 0.50, 0.65, 0.40, 0.60, 0.65, 0.5, 0.5, 0.5]
p0_std = [0.01, 0.01, 0.001, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.1, 0.1, 0.1]

p0 = np.random.normal(p0_base, p0_std, size=[ntemps, nwalkers, ndim])
print map(lnprior, p0[0])

In [ ]:
sampler = emcee.PTSampler(ntemps, nwalkers, ndim, triangle_llh, lnprior, threads=4)

In [ ]:
print "Running burn-in"
for result in tqdm.tqdm(sampler.sample(p0, iterations=burnin), total=burnin):
    pos, prob, state = result
sampler.reset()

In [ ]:
nsteps = 1000
width = 30

In [ ]:
for _ in tqdm.tqdm(sampler.sample(pos, iterations=nsteps), total=nsteps):
    pass

In [ ]:
samples = sampler.chain[0, :, :, :].reshape((-1, ndim))
print sampler.acceptance_fraction
print np.unique(samples[:,0]).shape